In [2]:
import pandas as pd
import numpy as np

# Đọc file CSV (dùng engine="python" và dtype=str để tránh lỗi do định dạng lạ)
df = pd.read_csv("news.csv", encoding="utf-8", engine="python", dtype=str)

# Tìm cột chứa 'content' (không phân biệt hoa thường)
content_col = None
for col in df.columns:
	if 'content' in str(col).lower():
		content_col = col
		break

if content_col is not None:
	sample_content = df.loc[0, content_col]
else:
	# Nếu không tìm thấy, thử lấy ô đầu tiên (có thể là JSON/string chứa content)
	first_cell = df.iloc[0, 0]
	try:
		import json
		parsed = json.loads(first_cell)
		if isinstance(parsed, dict):
			sample_content = parsed.get('content') or parsed.get('text') or first_cell
		else:
			sample_content = first_cell
	except Exception:
		sample_content = first_cell

print("Original Content:")
print(sample_content)


Original Content:
 "content": "Thời sự \n\n\nThứ hai


## Bước 2. Làm sạch văn bản (Cleaning)

Các bước dọn dẹp cơ bản:

Chuyển về chữ thường (lowercase).

Xóa dấu câu, số, ký tự đặc biệt.

Xóa khoảng trắng thừa.

In [3]:
import re

def clean_text(text):
    # Chuyển về chữ thường
    text = text.lower()
    
    # Xóa ký tự đặc biệt, số, dấu câu
    re.sub(r'[^a-zA-ZÀ-ỹ0-9%\s]', ' ', text)
    
    # Xóa khoảng trắng thừa
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

cleaned_content = clean_text(sample_content)
print("Cleaned Content:")
print(cleaned_content)


Cleaned Content:
"content": "thời sự \n\n\nthứ hai


## Bước 3. Tách từ (Tokenization)

Tiếng Việt cần tách từ đúng cách, ví dụ "bộ công thương" → "bộ_công_thương".
 dùng underthesea:
 ,hoặc 
 
 -pyvi

-VnCoreNLP

In [4]:

from underthesea import word_tokenize

tokenized_content = word_tokenize(cleaned_content, format="text")
print("Tokenized Content:")
print(tokenized_content)


Tokenized Content:
hai tàu cá do bà nguyễn_thị_bé , 54 tuổi , xã long_hải , tp hcm ( tỉnh bà rịa - vũng tàu cũ ) làm_chủ . sáng 28/9 , hai tàu cùng 13 ngư_dân vào trú_bão trên sông gianh , thuộc phường bắc_giang . mỗi tàu cử một ngư_dân ở lại trông_giữ tài_sản , số còn lại lên bờ chờ bão qua . theo báo_cáo của chính_quyền phường bắc_giang , khuya 28/9 thấy quảng_trị trời lặng_gió , cho rằng bão đã qua , các ngư_dân quay lại tàu kiểm_tra kỹ_thuật , cho nổ máy , nấu cơm và chuẩn_bị các nhu_yếu_phẩm chuẩn_bị ra biển . tuy_nhiên , đến 22 h30 , tâm_bão áp sát đất_liền quảng_trị - hà_tĩnh , gió_giật mạnh , mưa to , nước sông gianh chảy xiết khiến hai tàu bị đứt dây_neo , trôi tự_do . 4 ngư_dân bơi được vào bờ , hiện được đưa về hải_đội biên_phòng 1 chăm_sóc . 9 người còn mất_tích . lực_lượng chức_năng đang huy_động nhân_lực , phương_tiện tìm_kiếm . sông giang nước dâng cao , sáng 29/9 . ảnh : đắc_thành bão bualoi đổ_bộ vào đèo ngang , giáp_ranh quảng_trị - hà_tĩnh lúc 0 h30 hôm_nay . nhưng 6

## Bước 4. Loại bỏ Stopwords

Stopwords là các từ không mang nhiều thông tin, ví dụ: "là", "của", "đã", "và", "nhưng"...
Bạn có thể tự tạo một danh sách stopwords hoặc dùng có sẵn.

In [5]:
stopwords = ["là", "của", "đã", "và", "nhưng", "trong", "cùng"]

def remove_stopwords(text):
    words = text.split()
    filtered_words = [w for w in words if w not in stopwords]
    return " ".join(filtered_words)

filtered_content = remove_stopwords(tokenized_content)
print("Filtered Content (no stopwords):")
print(filtered_content)


Filtered Content (no stopwords):
hai tàu cá do bà nguyễn_thị_bé , 54 tuổi , xã long_hải , tp hcm ( tỉnh bà rịa - vũng tàu cũ ) làm_chủ . sáng 28/9 , hai tàu 13 ngư_dân vào trú_bão trên sông gianh , thuộc phường bắc_giang . mỗi tàu cử một ngư_dân ở lại trông_giữ tài_sản , số còn lại lên bờ chờ bão qua . theo báo_cáo chính_quyền phường bắc_giang , khuya 28/9 thấy quảng_trị trời lặng_gió , cho rằng bão qua , các ngư_dân quay lại tàu kiểm_tra kỹ_thuật , cho nổ máy , nấu cơm chuẩn_bị các nhu_yếu_phẩm chuẩn_bị ra biển . tuy_nhiên , đến 22 h30 , tâm_bão áp sát đất_liền quảng_trị - hà_tĩnh , gió_giật mạnh , mưa to , nước sông gianh chảy xiết khiến hai tàu bị đứt dây_neo , trôi tự_do . 4 ngư_dân bơi được vào bờ , hiện được đưa về hải_đội biên_phòng 1 chăm_sóc . 9 người còn mất_tích . lực_lượng chức_năng đang huy_động nhân_lực , phương_tiện tìm_kiếm . sông giang nước dâng cao , sáng 29/9 . ảnh : đắc_thành bão bualoi đổ_bộ vào đèo ngang , giáp_ranh quảng_trị - hà_tĩnh lúc 0 h30 hôm_nay . 6-8_tiến

## Bước 5. Vector hóa (TF-IDF)

Bây giờ, dữ liệu chữ đã sẵn sàng để biến thành vector số.

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
vector = tfidf.fit_transform([filtered_content])

# Xem các đặc trưng (từ vựng)
print("Tu_vung:", tfidf.get_feature_names_out())

# Xem vector TF-IDF
print("TF-IDF vector:\n", vector.toarray())

vocab = tfidf.get_feature_names_out()

df_vector = pd.DataFrame(vector.toarray(), columns=vocab)

df_vector.to_csv("du_lieu.csv", index=False, encoding="utf-8-sig")

print("Đã lưu vào file 'du_lieu.csv'")


Tu_vung: ['11' '13' '22' '28' '29' '54' '8_tiếng' 'biên_phòng' 'biển' 'bualoi' 'bà'
 'báo_cáo' 'bão' 'bơi' 'bắc_giang' 'bị' 'bờ' 'cao' 'cho' 'chuẩn_bị' 'chìm'
 'chính_quyền' 'chăm_sóc' 'chảy' 'chết' 'chờ' 'chức_năng' 'cá' 'các' 'còn'
 'có' 'cũ' 'cơm' 'cấp' 'cứu' 'cử' 'cửa' 'cửa_biển' 'do' 'dâng' 'dây_neo'
 'giang' 'gianh' 'giáp_ranh' 'gió' 'gió_giật' 'gồm' 'h30' 'hai' 'hcm'
 'hiện' 'huy_động' 'hà_tĩnh' 'hôm_nay' 'hải_đội' 'khiến' 'khuya'
 'kiểm_tra' 'kỹ_thuật' 'long_hải' 'làm_chủ' 'lên' 'lúc' 'lạch' 'lại'
 'lặng_gió' 'lực_lượng' 'máy' 'mưa' 'mạnh' 'mất_tích' 'mỗi' 'một' 'ngang'
 'nguyễn_thị_bé' 'ngư_dân' 'người' 'nhu_yếu_phẩm' 'nhân_lực' 'nước' 'nấu'
 'nổ' 'phương_tiện' 'phường' 'qua' 'quay' 'quảng_trị' 'ra' 'rằng' 'rịa'
 'sáng' 'sát' 'sóng' 'sông' 'số' 'theo' 'thuộc' 'thì' 'thấy' 'to' 'tp'
 'tránh' 'trên' 'trôi' 'trông_giữ' 'trú_bão' 'trước' 'trời' 'tuy_nhiên'
 'tuổi' 'tài_sản' 'tàu' 'tâm_bão' 'tìm_kiếm' 'tỉnh' 'tự_do' 'việt' 'vào'
 'vũng' 'về' 'xiết' 'xã' 'áp' 'đang' 'đánh' 'đèo' 'đó